# Logistic Regression Algorithm with Tensorflow

This is a simple logistic regression algorithm implemented using Tensorflow.
First, we will import the libraries needed for the algorithm.

In [1]:
import numpy as np
import tensorflow as tf
from sklearn.preprocessing import OneHotEncoder

Next, we will generate some data for the LR algorithm to work on. The first dataset consists of vectors in the form of [0, 1, 2,..., 19], normalized to the range of (0, 1), and then contaminated by random noise. The second dataset is made up of pure random noise.

In [2]:
# Generate two datasets
datagroup1 = (np.tile(np.arange(20),(1000,1))/19 + np.random.random((1000,20)))/2
datagroup2 = np.random.random((1000,20))
datagroup1_labeled = np.concatenate((datagroup1, np.ones((1000,1))), axis=1) 
datagroup2_labeled = np.concatenate((datagroup2, np.zeros((1000,1))), axis=1) 

Then, we concatenate the two dataset and shuffle them before feeding them to the LR algorithm. We also need to do some reshaping to make sure the data input is of the right dimensions.

In [3]:
alldata = np.concatenate((datagroup1_labeled, datagroup2_labeled), axis=0)
np.random.shuffle(alldata)
# The first 1500 entries are used as train data
x_train = alldata[0:1500,0:20]
y_train = alldata[0:1500,20]
# The next 500 entries are used as test data
x_test = alldata[1500:2000,0:20]
y_test = alldata[1500:2000,20]
# Reshape labels to meet Tensorflow requirements
y_train = np.reshape(y_train, [y_train.shape[0], 1])
y_test = np.reshape(y_test, [y_test.shape[0], 1])

We have to perform one-hot encoding to the labels.

In [4]:
# Creating the One Hot Encoder 
oneHot = OneHotEncoder() 
# Encoding labels
oneHot.fit(y_train) 
y_train = oneHot.transform(y_train).toarray() 
oneHot.fit(y_test) 
y_test = oneHot.transform(y_test).toarray() 

NameError: name 'oneHot' is not defined

The code to implement a LR model in Tensorflow is much chunkier than that in Keras. We first need to initialize placeholders for both input data and coefficients.

In [ ]:
# Create placeholders the size of data and label
X = tf.placeholder(tf.float32, [None, 20]) 
Y = tf.placeholder(tf.float32, [None, 2]) 
# Trainable variable weights 
W = tf.Variable(tf.zeros([20, 2])) 
# Trainable variable bias 
b = tf.Variable(tf.zeros([2])) 

Next, we need to define some hyperparameters (learning rate and the number of epochs to train).

In [ ]:
# Hyperparameters
lr, epochs = 0.0015, 100

For Tensorflow, we need to explictly define the math model used for LR, as well as the cost fundtion we wish to minimize.

In [ ]:
# Tensorflow implementation of LR
Y_hat = tf.nn.sigmoid(tf.add(tf.matmul(X, W), b)) 
  
# Sigmoid Cross Entropy Cost Function 
cost = tf.nn.sigmoid_cross_entropy_with_logits(logits = Y_hat, labels = Y) 

We then have to specify the optimizer, and initialize variables for the model.

In [ ]:
# Gradient Descent Optimizer 
optimizer = tf.train.AdamOptimizer(learning_rate = lr).minimize(cost) 
  
# Global Variables Initializer 
init = tf.global_variables_initializer() 

Tensorflow implementations goes by the name "sessions", we need to create a session for the training and test data, and display the training accuracy at the end.

In [ ]:
# Starting the Tensorflow Session 
with tf.Session() as sess: 
      
    # Initializing the Variables 
    sess.run(init) 
      
    # Lists for storing the changing Cost and Accuracy in every Epoch 
    cost_history, accuracy_history = [], [] 
      
    # Iterating through all the epochs 
    for epoch in range(epochs): 
        cost_per_epoch = 0
          
        # Running the Optimizer and cost 
        _, c = sess.run([optimizer, cost], feed_dict = {X : x_train, Y : y_train}) 
          
        # Calculating accuracy on current Epoch 
        correct_prediction = tf.equal(tf.argmax(Y_hat, 1), tf.argmax(Y, 1)) 
        accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32)) 
          
        # Storing Cost and Accuracy to the history 
        cost_history.append(sum(sum(c))) 
        accuracy_history.append(accuracy.eval({X : x_train, Y : y_train}) * 100) 
          
        # Displaying result on current Epoch 
        if epoch % 10 == 0 and epoch != 0: 
            print("Epoch " + str(epoch) + " Accuracy: " + str("%.1f" % accuracy_history[-1])  + "%") 
      
    Weight = sess.run(W) # Optimized Weight 
    Bias = sess.run(b)   # Optimized Bias 
      
    # Final Accuracy 
    correct_prediction = tf.equal(tf.argmax(Y_hat, 1), tf.argmax(Y, 1)) 
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32)) 
    trainacc = accuracy.eval({X : x_train, Y : y_train}) * 100
    testacc = accuracy.eval({X : x_test, Y : y_test}) * 100
    print("Train Accuracy:", str("%.1f" % trainacc), "%")
    print("Test Accuracy:", str("%.1f" % testacc), "%")